In [1]:
import os, sys
import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

from dask.distributed import Client, LocalCluster
from dask.base import compute
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, protocol = 'tcp://', host = '137.129.155.67')
client = Client(cluster)
client



/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44181 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:44181/status,
Dashboard: http://137.129.155.67:44181/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://137.129.155.67:35031,Workers: 1
Dashboard: http://137.129.155.67:44181/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://137.129.155.67:45463,Total threads: 4
Dashboard: http://137.129.155.67:45135/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/'
filenames = np.arange(1991,2020)
datasets = []
for f in filenames :
    _ds = xr.open_mfdataset(indir_data_FILTERED + 'TCWV/*' + str(f) + '*.nc', chunks = {'time' : 1}, parallel = True)
    _ds = _ds.sel(lat = slice(30.6,-30.6)).isel(lat = slice(0,None,16), lon = slice(0,None,16))
    datasets.append(_ds)
    
ds = xr.concat(datasets, dim = 'time') 
# ds = xr.open_mfdataset(indir_data_FILTERED + 'TCWV/*.nc', chunks = {'time' : 100}, parallel = True)
# ds = ds.sel(lat = slice(30,-30))
ds

<xarray.Dataset>
Dimensions:      (time: 84736, lat: 16, lon: 90)
Coordinates:
  * time         (time) datetime64[ns] 1991-01-01 ... 2019-12-31T21:00:00
  * lat          (lat) float64 30.5 26.5 22.5 18.5 ... -17.5 -21.5 -25.5 -29.5
  * lon          (lon) float32 0.0 4.0 8.0 12.0 16.0 ... 344.0 348.0 352.0 356.0
Data variables:
    TCWV_Kelvin  (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_Rossby  (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_MJO     (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_MRG     (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_EIG     (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_WIG     (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_TD      (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>
    TCWV_LF      (time, lat, lon) float64 dask.array<chunksize=(1, 16, 90), meta=np.ndarray>

In [3]:
ds_Var = ds.var(dim = 'time')
ds_Var = ds_Var.persist()

In [4]:
map_proj = ccrs.Mercator(central_longitude=180.0, 
                         min_latitude=-30.1, 
                         max_latitude=30.1,
                         globe=None)

# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = -10, +10

colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both'}


In [5]:
ds_plot = ds_Var

# Variance de la colonne d'eau filtré pour la MJO

In [ ]:
# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = 0, 10
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}
#####################################################################################

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_MJO'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MJO")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_TD'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour les TDs")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_Kelvin'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Kelvin")
plt.show()

#####################################################################################
levelsT = np.linspace(0,14,8)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_Rossby'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax+4,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Rossby")

plt.show()

#####################################################################################cmap ='BrBG'
vmin, vmax = 0, 5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_MRG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MRG")

plt.show()

#####################################################################################
cmap ='BrBG'
vmin, vmax = 0, 0.5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_EIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'EIG")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

ds_Var['TCWV_WIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la WIG")
plt.show()

# #####################################################

In [ ]:
months =  [1,2,3] # for example
_ds  =  ds.sel(time = np.in1d(ds['time.month'], months))
_ds_Var = _ds.var(dim = 'time')
_ds_plot = _ds_Var.load()

In [ ]:
# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = 0, 10
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}
#####################################################################################

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MJO'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MJO pour JFM")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_TD'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour les TDs pour JFM")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Kelvin'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Kelvin pour JFM")
plt.show()

#####################################################################################
levelsT = np.linspace(0,14,8)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Rossby'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax+4,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Rossby pour JFM")

plt.show()

#####################################################################################cmap ='BrBG'
vmin, vmax = 0, 5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MRG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MRG pour JFM")

plt.show()

#####################################################################################
cmap ='BrBG'
vmin, vmax = 0, 0.5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_EIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'EIG pour JFM")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_WIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la WIG pour JFM")
plt.show()

# #####################################################

In [ ]:
months =  [4,5,6] # for example
_ds  =  ds.sel(time = np.in1d(ds['time.month'], months))
_ds_Var = _ds.var(dim = 'time')
_ds_Var = _ds_Var
_ds_plot = _ds_Var.load()

In [ ]:
# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = 0, 10
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}
#####################################################################################

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MJO'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MJO pour AMJ")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_TD'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour les TDs pour AMJ")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Kelvin'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Kelvin pour AMJ")
plt.show()

#####################################################################################
levelsT = np.linspace(0,14,8)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Rossby'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax+4,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Rossby pour AMJ")

plt.show()

#####################################################################################cmap ='BrBG'
vmin, vmax = 0, 5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MRG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MRG pour AMJ")

plt.show()

#####################################################################################
cmap ='BrBG'
vmin, vmax = 0, 0.5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_EIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'EIG pour AMJ")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_WIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la WIG pour AMJ")
plt.show()

# #####################################################

In [ ]:
months =  [7,8,9] # for example
_ds  =  ds.sel(time = np.in1d(ds['time.month'], months))
_ds_Var = _ds.var(dim = 'time')
_ds_Var = _ds_Var
_ds_plot = _ds_Var.load()

In [ ]:
# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = 0, 10
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}
#####################################################################################

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MJO'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MJO pour JAS")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_TD'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour les TDs pour JAS")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Kelvin'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Kelvin pour JAS")
plt.show()

#####################################################################################
levelsT = np.linspace(0,14,8)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Rossby'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax+4,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Rossby pour JAS")

plt.show()

#####################################################################################cmap ='BrBG'
vmin, vmax = 0, 5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MRG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MRG pour JAS")

plt.show()

#####################################################################################
cmap ='BrBG'
vmin, vmax = 0, 0.5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_EIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'EIG pour JAS")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_WIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la WIG pour JAS")
plt.show()

# #####################################################

In [ ]:
months =  [10,11,12] # for example
_ds  =  ds.sel(time = np.in1d(ds['time.month'], months))
_ds_Var = _ds.var(dim = 'time')
_ds_Var = _ds_Var
_ds_plot = _ds_Var.load()

In [ ]:
# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = 0, 10
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}
#####################################################################################

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MJO'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MJO pour OND")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_TD'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour les TDs pour OND")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Kelvin'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Kelvin pour OND")
plt.show()

#####################################################################################
levelsT = np.linspace(0,14,8)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_Rossby'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax+4,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'onde de Rossby pour OND")

plt.show()

#####################################################################################cmap ='BrBG'
vmin, vmax = 0, 5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_MRG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la MRG pour OND")

plt.show()

#####################################################################################
cmap ='BrBG'
vmin, vmax = 0, 0.5
levelsT = np.linspace(0,vmax,6)
    
colorbar = {'label': 'TCWV $kg^{2}.m^{-4}$',
            'orientation': 'vertical' ,
            'extend' : 'both',
           'ticks': levelsT}

fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_EIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour l'EIG pour OND")

plt.show()

#####################################################################################
fig, axis = plt.subplots( 1, 1, 
    figsize=(20,4), subplot_kw={'projection': map_proj})

_ds_plot['TCWV_WIG'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.title("Variance de TCWV pour la WIG pour OND")
plt.show()